In [1]:
from fontTools.misc.classifyTools import Classifier
from sympy.tensor.tensor import tensor_mul

classification_nmaes={
    0: '上衣',
    1: '鞋子',
    2: '包',
    3: '下衣',
    4: '手表',
}

In [2]:
import torch
from PIL import Image
import torchvision.transforms as T
from torch.utils.data import Dataset
import os
import pandas as pd

def sorted_alphanumeric(data):
    import re
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

#定义自定义数据集类FolderDataset
class FolderDataset(Dataset):
    def __init__(self,main_dir,transform=None):
        self.root_dir=main_dir
        self.transform=transform
        self.all_image=sorted_alphanumeric(os.listdir(main_dir))
        self.classification_nmaes=pd.read_csv('../common/fashion-labels.csv')
        self.laberl_dict=dict(zip(self.classification_nmaes['id'],self.classification_nmaes['target']))  #拉链
    def __len__(self):
        return len(self.all_image)
    def __getitem__(self,idx):
        img_loc=os.path.join(self.root_dir,self.all_image[idx])

        image=Image.open(img_loc).convert('RGB')
        img_flag_id=self.labell_dict[idx]

        if self.transform:
            image=self.transform(image)
        else:
            raise RuntimeError("Transform is not defined")
        return tensor_image,img_flag_id


In [1]:
import re
import torch
import torchvision.transforms as T
#检查是否有可用的GPU,否则使用CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
transforms=T.Compose([
    T.Resize((64,64)),
    T.ToTensor(),

])
print('----------正在创建数据表----------')
full_dataset=FolderDataset('../common/dataset/', transform=transforms)
train_size=int(0.75*len(full_dataset))
val_size=len(full_dataset)-train_size
full_dataset,val_size=torch.utils.data.random_split(full_dataset,[train_size,val_size])


print('----------数据表创建完成----------')
print('----------正在创建数据加载器----------')

#定义批次大小
batch=32

#创建训练集的DataLoder,批次大小为32
train_loder=torch.utils.data.DataLoader(full_dataset,batch_size=batch,shuffle=True)
#创建验证集的DataLoder,批次大小为32
val_loader=torch.utils.data.DataLoader(val_size,batch_size=batch)

#创建完整集的DataLoder,批次大小为32
full_loder=torch.utils.data.DataLoader(full_dataset,batch_size=batch)

print('----------数据加载器创建完成----------')

#遍历训练集的DataLoder，打印每第一个批次的数据
for x,y in train_loder:
    print('图像张量形状:',x.shape)
    print('标签:',y.shape)
    break











----------正在创建数据表----------


NameError: name 'FolderDataset' is not defined

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1=nn.Linear(16*16*16,128)


    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(x)

        x=self.pool(F.relu(self.conv2(x)))
        x=self.pool(x)

        x=x.reshape(x.shape[0],-1)

        x=self.fc1(x)
        x=F.log_softmax(x,dim=1)
        return x



# 实例化CNN模型
model=ClassificationModel()

print(model)

#获取训练加载器的迭代器
train_iterator=iter(train_loder)
#获取下一个批次的数据
images,labels=next(train_iterator)
print('数据经过模型后向前传递后的维度:',model(images).shape)